# Session 9b - Dynamic topic modelling

# Data
This time, we're going to load an work with an English language novel. The goal of *dynamic topic modelling* is to study how topics evolve and interact over time.

In this case, we want to know how different topics evolve and change over the course of a single novel.

In [1]:
import os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
inpath = os.path.join("..", "data", "Cbronte_Jane_1847.txt")
with open(inpath, "r") as f:
    text = f.read()

In [4]:
print(text)

 Jane Eyre
by Charlotte Bronte
PREFACE
A preface to the first edition of "Jane Eyre" being unnecessary, I gave none: this second edition demands a few words both of acknowledgment and miscellaneous remark.
My thanks are due in three quarters.
To the Public, for the indulgent ear it has inclined to a plain tale with few pretensions.
To the Press, for the fair field its honest suffrage has opened to an obscure aspirant.
To my Publishers, for the aid their tact, their energy, their practical sense and frank liberality have afforded an unknown and unrecommended Author.
The Press and the Public are but vague personifications for me, and I must thank them in vague terms; but my Publishers are definite: so are certain generous critics who have encouraged me as only large-hearted and high-minded men know how to encourage a struggling stranger; to them, i.e., to my Publishers and the select Reviewers, I say cordially, Gentlemen, I thank you from my heart.
Having thus acknowledged what I owe tho

We're first going to do some *very* crude pre-processing by splitting the text up into sentences based on the presence of period/full stop.

**Question:** How might we improve this step?

In [5]:
docs = text.split(".")

In [7]:
len(docs)

8408

# **Topic Modeling**


## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language.

We will also calculate the topic probabilities which will be useful later.

**NB:** On a 4 CPU UCloud instance, this takes around 13 minutes to run!


In [8]:
# initialize the model
topic_model = BERTopic(language="english", 
                       calculate_probabilities=True, 
                       verbose=True)

# notice how we use the same fit_transform logic as we've seen before
topics, probs = topic_model.fit_transform(docs)

2024-04-04 11:08:43,480 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 263/263 [00:18<00:00, 14.60it/s]
2024-04-04 11:09:01,815 - BERTopic - Embedding - Completed ✓
2024-04-04 11:09:01,815 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-04 11:09:12,622 - BERTopic - Dimensionality - Completed ✓
2024-04-04 11:09:12,622 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-04 11:09:15,196 - BERTopic - Cluster - Completed ✓
2024-04-04 11:09:15,201 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-04 11:09:15,351 - BERTopic - Representation - Completed ✓


**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents. 

In [9]:
freq = topic_model.get_topic_info()
freq.head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4013,-1_to_the_and_of,"[to, the, and, of, you, it, my, in, me, was]","[\n""I did say so; and it is the name by which ..."
1,0,160,0_jane_you_me_your,"[jane, you, me, your, do, sir, are, voice, wil...","[ Jane - give it me now, ""\n""Now, Jane, that i..."
2,1,135,1_bread_eat_tea_tray,"[bread, eat, tea, tray, bell, breakfast, hungr...","[ I stopped and said -\n""Will you give me a pi..."
3,2,132,2_door_the_opened_staircase,"[door, the, opened, staircase, key, gallery, s...","[ Ere long, steps retreated up the gallery tow..."
4,3,128,3_his_eye_he_face,"[his, eye, he, face, eyes, features, looked, o...","[""\nWith this announcement he rose from his ch..."
5,4,123,4_adele_adeles_her_she,"[adele, adeles, her, she, with, to, for, madem...","[""\n""Adele, look at that field, Adele and Mrs..."
6,5,106,5_reed_reeds_her_she,"[reed, reeds, her, she, children, me, and, gat...","[ Reed, Reed, Reed]"
7,6,103,6_brother_sisters_uncle_sister,"[brother, sisters, uncle, sister, father, dead...","[ He was my mother's brother, \n""My uncle John..."
8,7,100,7_marry_wife_married_marriage,"[marry, wife, married, marriage, you, bride, u...","[ As to the new existence, it is all right: yo..."
9,8,100,8_mr_meantime_old_arrows,"[mr, meantime, old, arrows, stock, childer, tr...","[ Mr, \nMr, Mr]"


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [11]:
topic_model.get_topic(7)  # Select the most frequent topic

[('marry', 0.04291965721242369),
 ('wife', 0.03856101264534292),
 ('married', 0.035339044854272056),
 ('marriage', 0.02460424108725929),
 ('you', 0.017613651928811976),
 ('bride', 0.0151877982072687),
 ('union', 0.014802521074111447),
 ('your', 0.014681269540000546),
 ('will', 0.013761174943949921),
 ('be', 0.013111010952152394)]

__Get top 10 topics__

In [12]:
topic_model.topics_[:10]

[11, -1, -1, -1, -1, -1, 74, -1, -1, -1]

# **Visualization**

## Visualize Topics

In [13]:
topic_model.visualize_topics()

## Visualize Topic Probabilities

In [14]:
topic_model.visualize_distribution(probs[10])

## Visualize Topic Hierarchy

In [34]:
topic_model.visualize_hierarchy(top_n_topics=20)

## Visualize Terms

In [28]:
topic_model.visualize_barchart(top_n_topics=20)

## Visualize Topic Similarity

In [39]:
topic_model.visualize_heatmap(n_clusters=50, 
                              width=1000, 
                              height=1000)

# **Search Topics**

In [ ]:
similar_topics, similarity = topic_model.find_topics("love", top_n=5)
print(similar_topics)

In [ ]:
topic_model.get_topic(5)

In [ ]:
topic_model.get_representative_docs(5)

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created. 

This allows for fine-tuning the model to your specifications and wishes. 

## Update Topics

One of the easiest ways to make our topics more coherent is to revise the *topic descriptions* rather than training a new model. 

This means that we try to foreground more potentially significant and meaningful words for individual topics.

We can do this by creating a simple bag-of-words model and filtering based on some conditions - just like we do when working with scikit-learn. 

**NB:** This will also take around 25 mins to run on a 4 CPU machine.

In [27]:
vectorizer_model = CountVectorizer(stop_words="english", 
                                   ngram_range=(1, 2),
                                   min_df=0.05,
                                   max_df=0.9)
topic_model.update_topics(docs, vectorizer_model=vectorizer_model)

In [ ]:
topic_model.get_topic(1)   # We select topic that we viewed before

## Dynamic topic models

We first need to create some kind of *timestamps* for our data.

**Question:** In this context, what might be a good idea for creating timestamps? What is this cell below doing?

In [29]:
timestamps = list(range(len(docs)))

In [30]:
topics_over_time = topic_model.topics_over_time(docs, 
                                                timestamps, 
                                                nr_bins=10)

10it [00:00, 30.82it/s]


__Plotting topics over time__

In [33]:
topic_model.visualize_topics_over_time(topics_over_time, 
                                       normalize_frequency=True, 
                                       top_n_topics=20)

__Plot specific topics__

In [32]:
topic_model.visualize_topics_over_time(topics_over_time, 
                                       normalize_frequency=True, 
                                       topics=[12]) # list of chosen topics